# It's a big To-Do List

Harry says the purpose of the website in this example will be pretty straightforward: a website hosting a list of actionable items.

<div class="alert alert-info">There's a box here saying that what he calls a <b>Functional Test</b> is sometimes called by others an <b>Acceptance Test</b> or <b>End-to-End Test</b>, or even <b>Black Box Test</b>, and that we should be aware that all of these are interchangeable terms, at least for our purposes here.</div>

One concept is to consider such functional tests a human-readable "user story", i.e., patterned on how an end user might interact with our software in a common use case, from start to finish.  He starts with adding a bunch of comments to the `functional_tests.py` file written in the last chapter, but then points out they're largely overkill, so I'll skip most here.  But we do at least update what the title of the browser window should be:

In [4]:
%%writefile functional_tests.py

from selenium import webdriver

browser = webdriver.Firefox()

# Edith wants to use our to-do page, and notes the url:
browser.get('http://localhost:8000')

# Its purpose is to host a to-do list
assert 'To-Do' in browser.title

# Additional downstream tests to ensure the server stays up and 
# saves changes made by the user to the list...

browser.quit()

Overwriting functional_tests.py


Ok, then we can (again) launch the server with

`python manage.py runsever`

But not from within this notebook, with the way my whole Windows/Firefox/WSL setup is currently rigged.  Once that's being hosted, running 

`python functional_tests.py`

in another tab/window should generate an `AssertionError`, which it does for me at present.  Harry notes that's called an *expected fail*; a pretty minor first demonstration, but important to establish, nonetheless.

# The Python Standard Library's `unittest` Module

Harry notes that one way to get a more informative expected failure than the blank `AssertionError` by just instructing the interpreter to spit out more of what it's seeing, with a line like:

`assert 'To-Do' in browser.title, "Browser title was " + browser.title`

and, since it's annoyint to leave Firefox open after this basic test fails, you could imagine using a `try`/`except`/`finally` block to handle closing the browser even if the assertion failure interrupts your script.  But these are such common needs that they're more professionally handled by whole packages/modules, including the `unittest` module that comes with every Python installation.  So we update the `functional_tests.py` file accordingly:

In [5]:
%%writefile functional_tests.py

from selenium import webdriver
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        self.fail('Finish the test!')

        # She is invited to enter a to-do item straight away
        # ...
    
if __name__ == '__main__':
    unittest.main(warnings='ignore')

Overwriting functional_tests.py


For me, that outputs:

~~~bash
(test_dev_book) C:\Data\projects\test_driven_development>python functional_tests.py
F
======================================================================
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 19, in test_can_start_a_list_and_retrieve_it_later
    def tearDown(self):
AssertionError: 'To-Do' not found in 'Welcome to Django'

----------------------------------------------------------------------
Ran 1 test in 6.532s

FAILED (failures=1)
~~~


But before even evaluating outputs, he notes things about the code itself:

* You're subclassing `unittest.TestCase`
* The obnoxiously-long function name above is doing most of the heavy lifting there; apparently any such function within your subclass that starts with `test_` will be recognized and run by `unittest` whenever your `main` method is run.
    * I'd personally abbreviate that method's name, but that's an aesthetic judgment; the important thing is to keep it sufficiently informative.
* The `setUp` and `tearDown` methods are recognized by UnitTest to be executed sort of like the aforementioned `try`/`finally` kind of logic.  The result is that the Firefox window spawned is closed by the program.
    * I also don't love the method names here; camel case seems more old-fashioned than lowercase with underscores, but I don't use classes a lot to date, perhaps that's still recommended.  And anyways you have to stick with what the module will recognize.
* `unittest.TestCase.assertIn` replaces the builtin `assert` statement and accordingly brings in more helper functionality that is appropriate for tests; he points us to [the docs](https://docs.python.org/3/library/unittest.html) for details.
* The `self.fail` method is meant as a placeholder to remind you to add more to your code base; it will always produce the failure and error message passed to it.